In [1]:
import pandas as pd
import os
import sys
import json
import numpy as np
import tensorflow as tf
import keras.layers as L
import keras.activations as A
import keras.optimizers as O
import keras.losses as Loss
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras import backend as KB
from keras.callbacks import Callback
import time
from fc_conv import FC_CONV
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# sess = tf.Session(config=config)
# sess.run(tf.global_variables_initializer())

/home/ai-trainingmachine/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
ROOTDIR=os.path.abspath("../../")
sys.path.append(ROOTDIR)

In [3]:
from Preprocessing.Preprocessor import preprocessor
from data_utils.keras_generator_stratified import Generator
from utils.losses import weighted_cross_entropy_loss,f1_score_metric
from utils.callbacks import get_tensorboard_callback,get_checkpoint_call_back
from utils.tools import construct_model_from_csv
from models.BaseModel import Base_Model

In [4]:
label_encoder_dir=os.path.join(ROOTDIR,"Preprocessing")
data_dir="/home/ai-trainingmachine/Desktop/internship.ai/New_Data"#os.path.join(ROOTDIR,"Data")

In [5]:
files=os.listdir(data_dir)
array=pd.read_hdf(os.path.join(data_dir,files[0]))

P=preprocessor(label_encoder_dir)
array=P.process_batch(array)
action_col=array.columns=='action'

g_config={'labels':6,
        'weightage':[1,1,1,1,1,1],
        'data_directory':data_dir,
        'file_batch_size':1000,
        'format_':'h5',
        'filepath':"/home/ai-trainingmachine/Desktop/Usman/internship.ai/Train.npy",
        'ex_batch_size':1000,
        'file_examples':5000,
        'max_queue_size':1,
        'Preprocessor':P,
        'static_weightage':False,
         'action_col':action_col}

/home/ai-trainingmachine/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
class FC_CONV_Generator(Generator):
    def __init__(self,config):
        super().__init__(config)
    
    def __getitem__(self, idx):
        """
        fetches items from the super class and converts them to the appropriate format
        the super class returns inputs and  targets
        inputs contain the follwoing things in order
            id
            lane_change_mode
            ObNet (flat) (col no 2 to 467)
            SafeNetFront
            Speed
            Speed Limit
            PreviousDecision
            Previous_Speed
        targets contain the actions (label encoded)
        
        """
        x,targets=super().__getitem__(idx)
        ob_net=(x[:,2:467]).reshape(-1,155,3)
        num_cols=x.shape[1]
        
        scalar_features_col_mask=np.arange(num_cols)
        scalar_features_col_mask=(scalar_features_col_mask<2 ) |(scalar_features_col_mask>=467 )
        scalar_features=x[:,scalar_features_col_mask]
        
        return [ob_net,scalar_features],targets

In [7]:
G=FC_CONV_Generator(config=g_config)
G.load_from_file()
# G.generate()

In [8]:
input_shape=[(155,3),(7,)]
num_outputs=g_config['labels']
optimizer=O.Adam(lr=0.0001)
design_config_files=["conv_net.csv","fc_net.csv","merged_net.csv"]
model=FC_CONV(input_shape=input_shape,num_outputs=num_outputs,
        optimizer=optimizer,arch_config_files=design_config_files)
model.construct_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 155, 3, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 155, 3, 64)   640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 155, 3, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 155, 3, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

/home/ai-trainingmachine/Desktop/Armughanisback/internship.ai-feature-FC_CONV_model/models/fc_conv/fc_conv.py:48: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  self.Model=Model(inputs=[conv_input,scalar_features],output=logits)


In [9]:
KB.tensorflow_backend._get_available_gpus()


['/gpu:0']

In [10]:
model.train_generator(G,epochs=10,save_N_epochs=1)

Epoch 1/10
5000/5000 [==============================] - 776s 155ms/step - loss: 0.2521 - acc: 0.9020 - f1_calc: 0.8998

Epoch 00001: loss improved from inf to 0.25210, saving model to ../../checkpoints/FC_CONV_chkpoint__01.hdf5
Epoch 2/10
5000/5000 [==============================] - 682s 136ms/step - loss: 0.0595 - acc: 0.9770 - f1_calc: 0.9769

Epoch 00002: loss improved from 0.25210 to 0.05949, saving model to ../../checkpoints/FC_CONV_chkpoint__02.hdf5
Epoch 3/10
5000/5000 [==============================] - 682s 136ms/step - loss: 0.0402 - acc: 0.9828 - f1_calc: 0.9827

Epoch 00003: loss improved from 0.05949 to 0.04015, saving model to ../../checkpoints/FC_CONV_chkpoint__03.hdf5
Epoch 4/10
5000/5000 [==============================] - 687s 137ms/step - loss: 0.0370 - acc: 0.9833 - f1_calc: 0.9833

Epoch 00004: loss improved from 0.04015 to 0.03701, saving model to ../../checkpoints/FC_CONV_chkpoint__04.hdf5
Epoch 5/10
 523/5000 [==>...........................] - ETA: 10:26 - loss: 0

KeyboardInterrupt: 

In [11]:
model.Model.save_weights("a.h5")